In [1]:
# generated by gpt
person_data = [
    {
        "姓名": "林浩宇",
        "性别": "男",
        "年龄": 32,
        "身份证号": "310104199305187632",
        "手机号": "13817569241",
        "邮箱": "linhaoyu88@163.com",
        "住址": "上海市徐汇区宜山路455弄12号702室",
        "职业": "软件工程师",
        "学历": "本科",
        "毕业院校": "上海交通大学",
        "出生日期": "1993-05-18",
        "婚姻状况": "已婚",
        "紧急联系人": "陈梦琪",
        "紧急联系电话": "13916728503",
    },
    {
        "姓名": "张雅琪",
        "性别": "女",
        "年龄": 27,
        "身份证号": "440106199811035247",
        "手机号": "13570286419",
        "邮箱": "zhangyaqi@qq.com",
        "住址": "广州市天河区天河路385号1504房",
        "职业": "市场营销专员",
        "学历": "硕士",
        "毕业院校": "中山大学",
        "出生日期": "1998-11-03",
        "婚姻状况": "未婚",
        "紧急联系人": "张明远",
        "紧急联系电话": "13725690138",
    },
    {
        "姓名": "王建国",
        "性别": "男",
        "年龄": 55,
        "身份证号": "110102197003251879",
        "手机号": "13901087654",
        "邮箱": "wangjianguo65@sohu.com",
        "住址": "北京市西城区复兴门内大街15号6单元301",
        "职业": "退休教师",
        "学历": "大专",
        "毕业院校": "北京教育学院",
        "出生日期": "1970-03-25",
        "婚姻状况": "已婚",
        "紧急联系人": "刘桂芬",
        "紧急联系电话": "13681452907",
    },
    {
        "姓名": "陈佳怡",
        "性别": "女",
        "年龄": 24,
        "身份证号": "330105200107124563",
        "手机号": "13758190362",
        "邮箱": "chenjiayi2001@outlook.com",
        "住址": "杭州市西湖区文三路259号8幢2单元501",
        "职业": "护士",
        "学历": "专科",
        "毕业院校": "浙江医学高等专科学校",
        "出生日期": "2001-07-12",
        "婚姻状况": "未婚",
        "紧急联系人": "陈志强",
        "紧急联系电话": "13867345219",
    },
    {
        "姓名": "赵宇轩",
        "性别": "男",
        "年龄": 38,
        "身份证号": "510104198709086321",
        "手机号": "13688097542",
        "邮箱": "zhaoyuxuan@126.com",
        "住址": "成都市锦江区红星路三段1号12栋3单元1803",
        "职业": "建筑设计师",
        "学历": "本科",
        "毕业院校": "西南交通大学",
        "出生日期": "1987-09-08",
        "婚姻状况": "已婚",
        "紧急联系人": "李雪",
        "紧急联系电话": "13547628901",
    },
    {
        "姓名": "刘梦涵",
        "性别": "女",
        "年龄": 42,
        "身份证号": "210103198304158765",
        "手机号": "13840123567",
        "邮箱": "liumeng han83@163.com",
        "住址": "沈阳市和平区南京南街156号7单元402",
        "职业": "会计",
        "学历": "本科",
        "毕业院校": "东北财经大学",
        "出生日期": "1983-04-15",
        "婚姻状况": "离异",
        "紧急联系人": "刘梦琪",
        "紧急联系电话": "13904056782",
    },
    {
        "姓名": "黄梓豪",
        "性别": "男",
        "年龄": 29,
        "身份证号": "440306199612253698",
        "手机号": "13510876249",
        "邮箱": "huangzihao96@gmail.com",
        "住址": "深圳市南山区科技园南区10栋5单元2307",
        "职业": "产品经理",
        "学历": "硕士",
        "毕业院校": "华南理工大学",
        "出生日期": "1996-12-25",
        "婚姻状况": "已婚",
        "紧急联系人": "吴雨桐",
        "紧急联系电话": "13698754320",
    }
]

In [2]:
len(person_data)

7

In [8]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
# from dotenv import load_dotenv
import os

# 加载环境变量（建议将数据库连接信息放在 .env 文件中，避免硬编码）
# load_dotenv()  # 读取 .env 文件
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017/")  # 默认本地连接
DB_NAME = "person_db"  # 数据库名
COLLECTION_NAME = "persons"  # 集合名（类似表）


In [11]:
def init_database():
    """初始化数据库，创建集合并为 name 字段添加唯一索引"""
    # 连接 MongoDB
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    
    # 为 name 字段创建唯一索引（关键：确保 name 不重复）
    # 若索引已存在，此操作会忽略，不会报错
    collection.create_index("姓名", unique=True)
    
    return collection

In [12]:
collection = init_database()

In [13]:
def insert_person(collection, **kwargs):
    """
    插入人物信息，若 name 已存在则拒绝写入
    
    参数：
        collection: MongoDB 集合对象
        name: 人物姓名（唯一）
        age: 年龄
        **kwargs: 其他可选字段（如 gender, address 等）
    返回：
        成功返回插入的文档ID，失败返回错误信息
    """
    try:
        # 插入文档
        result = collection.insert_one(kwargs)
        return f"插入成功，ID: {result.inserted_id}"
    except DuplicateKeyError:
        # 捕获 name 重复的异常
        return f"拒绝写入：name '{kwargs}' 已存在"

In [14]:
# insert all data
for item in person_data:
    insert_person(collection, **item)

In [15]:
# query item
def query_by_name(collection, name):
    """
    根据姓名精确查询人物信息
    
    参数：
        collection: MongoDB 集合对象
        name: 要查询的姓名
    返回：
        符合条件的文档，如果不存在则返回None
    """
    result = collection.find_one({"姓名": name})
    # if result:
    #     result["_id"] = str(result["_id"])
    return result

In [22]:
obj = query_by_name(collection, "刘梦涵")
obj

{'_id': ObjectId('689afe0ea1a00a10e09fc4c1'),
 '姓名': '刘梦涵',
 '性别': '女',
 '年龄': 42,
 '身份证号': '210103198304158765',
 '手机号': '13840123567',
 '邮箱': 'liumeng han83@163.com',
 '住址': '沈阳市和平区南京南街156号7单元402',
 '职业': '会计',
 '学历': '本科',
 '毕业院校': '东北财经大学',
 '出生日期': '1983-04-15',
 '婚姻状况': '离异',
 '紧急联系人': '刘梦琪',
 '紧急联系电话': '13904056782'}